In [7]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [8]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in the field of natural language processing (NLP) and have numerous applications in various industries. Here are some reasons why fast language models are important:

1. **Improved User Experience**: Fast language models enable real-time language understanding and generation, which is essential for applications like chatbots, virtual assistants, and language translation software. Quick responses improve the overall user experience and make interactions more engaging.
2. **Efficient Processing of Large Volumes of Data**: Language models need to process vast amounts of text data, which can be computationally expensive. Fast language models can handle large volumes of data efficiently, reducing the processing time and enabling applications like text classification, sentiment analysis, and information retrieval.
3. **Real-time Language Translation**: Fast language models are essential for real-time language translation applications, such as live subtitles, 

In [9]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message: 
            self.messages.append({"role":"user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            messages = self.messages,
            model = "llama3-70b-8192"
        )
        return completion.choices[0].message.content

In [10]:
system_prompt = """

You run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you habe been asked.
Use Action to run one of the action available to you - then return PAUSE.
Observation will be the result of running those actions.

You available actions are:

calculat:
e.g. calculate: 4 * 7 / 3
Runs a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I neet to fin te mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this: 

Observation: 5.972e24

Thought: I need to multiply this by 2 
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 5.972e24

Thoutht: I need to multiply this by 2 
Action: calculate  5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1.1944x10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,944x10e25. 

Now its your turn:

""".strip()

In [11]:
# Ferramentas (Tools)
def calculate(operation):
    return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [12]:
neil_tyson = Agent(client, system_prompt)

In [13]:
result = neil_tyson("what is the mass of Earth times 5?")
print(result)

Thought: I need to find the mass of Earth and then multiply it by 5.


In [14]:
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I neet to fin te mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2 \nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThoutht: I need to mul

In [15]:
result = neil_tyson()
print(result)

In [16]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [17]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: I need to multiply this by 5.

Action: calculate: 5.972e24 * 5
PAUSE


In [18]:
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I neet to fin te mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2 \nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThoutht: I need to mul

In [21]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break



In [24]:
agent_loop(10, system = system_prompt, query="what is the mass of the Earth plus the mass of Mercury, and All of it times 5")

Thought: I need to find the mass of Earth and Mercury, and then add them together, and finally multiply the result by 5.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now I have the mass of Earth, I need to get the mass of Mercury
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: Now I have the mass of Earth and Mercury, I need to add them together
Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now I have the sum of the masses of Earth and Mercury, I need to multiply it by 5
Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have the final result 

Answer: The mass of the Earth plus the mass of Mercury, and all of it times 5 is 3.15025e+25.
